___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Copyright by Pierian Data Inc.</em></center>
<center><em>For more information, visit us at <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Selling Securities

Of course someday we need to liquidate (exit our positions and sell in this case of holding stock) our equities, or sell some part of them.
There are multiple methods to sell stocks:
1. Sell at a target price
2. Sell after some specific amount of time has passed
3. Sell after a certain percent gain.

**Important Note: the Liquidate() call is actually a general call for exiting a position. Which means if you are short a stock, it can still be used for liquidating your position by buying back shares. It is not technically the same as "selling", but instead, think of it as "closing out" a position.**

## Selling at a target price

**NOTE: In reality, we would never really sell at a *hard-coded* price! This is merely to illustrate how to compare to some variable price!**

To do so, we can simply check for the current price and use the **Liquidate** functionality, which sells all stocks of a given symbol and use a toggle *invest* to decide whether we want to buy or not

In [ ]:
class SellAtPrice(QCAlgorithm):

    def Initialize(self):
        
        self.SetCash(10000)
        self.SetStartDate(2010,1,1) 
        self.SetEndDate(2020,1,1)
        self.apple = self.AddEquity("AAPL", Resolution.Daily)
        self.limit_price = 50
        self.invest = True
        
    def OnData(self, data):
        if not self.Portfolio.Invested and self.invest:  # If not invested, invest everything in Apple
            self.SetHoldings("AAPL", 1)
        
      # If invested and the current apple price is > 50$, sell everything
      # update invest so we dont buy again
        closing_price = self.Portfolio["AAPL"].Price
        if closing_price > 50 and self.Portfolio.Invested:
            self.Liquidate("AAPL")
            self.invest=False


## Selling after some specific amount of time has passed


In [ ]:
class SellAfterTime(QCAlgorithm):

    def Initialize(self):
        
        self.SetCash(10000)
        self.SetStartDate(2010,1,1) 
        self.apple = self.AddEquity("AAPL", Resolution.Daily)
        self.invest = True # without this variable, we would rebuy AAPL after selling
        
        
    def OnData(self, data):
        if not self.Portfolio.Invested and self.invest:  # If not invested, invest everything in Apple
            self.SetHoldings("AAPL", 1)
            self.invested_time = self.Time
           
        
      # as the self.Time object in quantconnect is a python datetime object, we can simply substract
      # the current time from the invest time and sell if e.g 1000 days have passed
        self.Log(self.Time - self.invested_time)
        time_diff = (self.Time - self.invested_time).days
        if time_diff > 1000:
            self.Liquidate("AAPL")
            self.invest=False


## Selling after a profit threshold has been reached

In [ ]:
class SellAfterPercent(QCAlgorithm):

    def Initialize(self):
        
        self.SetCash(10000)
        self.SetStartDate(2010,1,1) 
        self.SetStartDate(2015,1,1)
        self.apple = self.AddEquity("AAPL", Resolution.Daily)
        self.limit_profit_percent = 1
        self.invest = True
        
        
    def OnData(self, data):
        
        if not self.Portfolio.Invested and self.invest:  # If not invested, invest everything in Apple
            self.SetHoldings("AAPL", 1) 
        
        # Grab Portfolio Attributes
        
        # Use for raw dollar profit
        # profitDollarValue = self.Portfolio[symbol].UnrealizedProfit
        
        # Use for profit in terms of percent (more realistic)
        # profitPercentage  = self.Portfolio[symbol].UnrealizedProfitPercent
        
        profit_percent  = self.Portfolio['AAPL'].UnrealizedProfitPercent 
        
        if profit_percent > self.limit_profit_percent:
            self.Liquidate("AAPL")
            self.invest=False

## Sell after Loss Limit Reached

### NOTE! We switched from Apple (AAPL) to American Airlines (AAL) to make sure we experience a loss. Also check out the start/end dates.

In [ ]:
class SellAfterPercent(QCAlgorithm):

    def Initialize(self):
        
        self.SetCash(10000)
        self.SetStartDate(2018,1,1) 
        self.SetEndDate(2021,1,1) 
        self.apple = self.AddEquity("AAL", Resolution.Daily)
        self.limit_loss_percent = -0.5
        self.invest = True
        
        
    def OnData(self, data):
        
        if not self.Portfolio.Invested and self.invest:  # If not invested, invest everything in Apple
            self.SetHoldings("AAL", 1) 
        
        # Grab Portfolio Attributes
        
        # Use for raw dollar profit
        # profitDollarValue = self.Portfolio[symbol].UnrealizedProfit
        
        # Use for profit in terms of percent (more realistic)
        # profitPercentage  = self.Portfolio[symbol].UnrealizedProfitPercent
        
        profit_percent  = self.Portfolio['AAL'].UnrealizedProfitPercent  
        
        # Sell if we lose more than 50% of our initial value
        if profit_percent < self.limit_loss_percent:
            self.Liquidate("AAL")
            self.invest=False